In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get Started with Gemini Preference Optimization

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_get_started.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ftuning%2Fdpo_gemini_get_started.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/tuning/dpo_gemini_get_started.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_get_started.ipynb">
      <img width="32px" src="https://storage.googleapis.com/github-repo/generative-ai/logos/GitHub_Invertocat_Dark.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

| Authors |
| --- |
| James Su |
| [Ivan Nardini](https://github.com/inardini) |

## Overview

This tutorial shows you how to teach Gemini to generate better responses by showing it examples of what humans prefer.

### What is Preference Optimization?

Instead of labeling responses as "correct" or "incorrect," preference optimization works with human preferences. You show the model pairs of responses to the same question—one that humans preferred and one they didn't—and the model learns to generate responses more like the preferred ones.

Think of it like this: rather than teaching a student the "right answer," you're showing them two essays and saying "this style is better than that style."

### What you'll learn

By the end of this tutorial, you will:
1. Load a dataset with human preference ratings
2. Train Gemini to align with those preferences
3. See a clear before-and-after comparison of model outputs

---

**Note**: This tutorial uses experimental features. They work great for learning and prototyping, but check the latest docs before using in production.

## Get started

### Prerequisites

- A Google Cloud project with billing enabled
- The Vertex AI API enabled ([enable it here](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com))
- No machine learning experience required!


### Install required packages

We'll install three packages:
- `google-genai`: The Gemini SDK for Python
- `google-cloud-aiplatform`: Vertex AI client library
- `datasets`: HuggingFace library to easily load training data

**⚠️ Expected Behavior**: After running this cell, Colab will ask you to **restart the runtime**. This is normal and necessary—click the "Restart Runtime" button when you see it.

In [ ]:
%pip install --upgrade --quiet google-genai google-cloud-aiplatform datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 992.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


### Authenticate (Colab only)

If you're running this in Google Colab, you need to authenticate so the notebook can access your Google Cloud project. This cell will prompt you to sign in.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print("✅ Authentication successful!")

✅ Authentication successful!


### Set up your Google Cloud project

Replace `[your-project-id]` below with your actual Google Cloud project ID. You can find this in the [Google Cloud Console](https://console.cloud.google.com/).

In [ ]:
import os

from google import genai
from google.genai import types

# TODO: Replace with your actual project ID
# fmt: off
PROJECT_ID = "cloud-llm-preview1"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on

# Auto-detect from environment if not set
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT", ""))

if not PROJECT_ID:
    raise ValueError("Please set your PROJECT_ID above")

# Region where the model will be tuned (us-central1 has the best availability)
LOCATION = "us-central1"

# Create a unique bucket name using your project ID
# Bucket names must be globally unique across all of Google Cloud
BUCKET_NAME = f"{PROJECT_ID}-preference-tuning"
BUCKET_URI = f"gs://{BUCKET_NAME}"

print(f"📦 Creating GCS bucket: {BUCKET_NAME}...")

# Create the bucket (the 2>/dev/null hides the error if it already exists)
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI} 2>/dev/null || echo "(Bucket already exists, continuing...)"

# Initialize the Gemini client with your project
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Using project: {PROJECT_ID}")
print(f"✅ Using region: {LOCATION}")
print(f"✅ Bucket: {BUCKET_URI}")
print("✅ Client initialized")

📦 Creating GCS bucket: cloud-llm-preview1-preference-tuning...
(Bucket already exists, continuing...)
✅ Using project: cloud-llm-preview1
✅ Using region: us-central1
✅ Bucket: gs://cloud-llm-preview1-preference-tuning
✅ Client initialized


### Import libraries

Now we'll import the Python libraries we need and set up the Gemini client.

In [ ]:
import json

from IPython.display import Markdown, display
from datasets import load_dataset

print("✅ Libraries imported")

✅ Libraries imported


## Step 1: Load the training dataset

We'll use **[UltraFeedback](https://huggingface.co/datasets/zhengr/ultrafeedback_binarized)**, a dataset containing thousands of prompts with multiple AI-generated responses rated by humans. For each prompt, humans indicated which response they preferred.

This is perfect for teaching Gemini what kind of responses humans find more helpful.

**What's in this dataset?**
- `prompt`: The user's question
- `chosen`: The response humans preferred (scored higher)
- `rejected`: The response humans didn't prefer (scored lower)
- `score_chosen` and `score_rejected`: The numerical ratings

In [ ]:
print("📥 Loading UltraFeedback dataset from HuggingFace...")
print("(This may take 1-2 minutes on first load)\n")

# Load the dataset
# The dataset has already been processed to identify preferred vs rejected responses
dataset = load_dataset("zhengr/ultrafeedback_binarized")

print("✅ Dataset loaded!\n")
print(f"Training examples: {len(dataset['train_prefs']):,}")
print(f"Test examples: {len(dataset['test_prefs']):,}")

📥 Loading UltraFeedback dataset from HuggingFace...
(This may take 1-2 minutes on first load)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train_sft-00000-of-00001-49a5a3acc4(…):   0%|          | 0.00/222M [00:00<?, ?B/s]

data/test_sft-00000-of-00001-0813b122475(…):   0%|          | 0.00/3.50M [00:00<?, ?B/s]

...

Generating train_gen split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_prefs split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Dataset loaded!

Training examples: 61,966
Test examples: 2,000


Let's look at one example to understand the structure:

In [ ]:
# Grab the first example from the training set
sample = dataset["train_prefs"][0]

print("📋 Example from the dataset:\n")
print(f"Prompt: {sample['prompt']}\n")
print(f"Chosen score: {sample.get('score_chosen', 'N/A')}")
print(f"Rejected score: {sample.get('score_rejected', 'N/A')}\n")
print("Full structure:")
print(json.dumps(sample, indent=2)[:1200] + "\n...\n")

📋 Example from the dataset:

Prompt: Write a 1,000-word op-ed piece in a formal tone, analyzing and providing examples of the ways in which social media platforms have been utilized to spread extremist and violent ideologies. In your analysis, discuss the specific tactics that these groups use to spread their messages online and the effects of these tactics on both individuals and society. Additionally, provide possible solutions that could be implemented to combat the spread of these dangerous ideologies on social media. Your piece should be well-researched, citing reputable sources to support your arguments.

Chosen score: 7.0
Rejected score: 6.0

Full structure:
{
  "prompt": "Write a 1,000-word op-ed piece in a formal tone, analyzing and providing examples of the ways in which social media platforms have been utilized to spread extremist and violent ideologies. In your analysis, discuss the specific tactics that these groups use to spread their messages online and the effects of th

## Step 2: Transform the data for Gemini

The dataset is in a generic format, but Gemini's tuning API expects a specific structure. We need to transform each example.

**From UltraFeedback format:**

```json
{
  "chosen": [{"role": "user", "content": "..."},
             {"role": "assistant", "content": "..."}],
  "rejected": [{"role": "user", "content": "..."},
               {"role": "assistant", "content": "..."}]
}
```

**To Gemini format:**

```json
{
  "contents": [{"role": "user", "parts": [{"text": "..."}]}],
  "completions": [
    {"score": 1.0, "completion": {"role": "model", "parts": [{"text": "..."}]}},
    {"score": 0.0, "completion": {"role": "model", "parts": [{"text": "..."}]}}
  ]
}
```

We'll write a helper function to do this transformation:

In [ ]:
def transform_to_gemini_format(example: dict) -> dict:
    """Convert one UltraFeedback example to Gemini's preference tuning format.

    Args:
        example: A dict with 'chosen' and 'rejected' fields

    Returns:
        A dict in Gemini's format with 'contents' and 'completions'
        None if the example is invalid
    """
    try:
        # Access the fields directly
        chosen = example["chosen"]
        rejected = example["rejected"]

        # Validate: need at least 2 messages in each
        if not isinstance(chosen, list) or len(chosen) < 2:
            return None
        if not isinstance(rejected, list) or len(rejected) < 2:
            return None

        # Extract the user prompt (first message)
        user_prompt = chosen[0].get("content", "").strip()

        # Extract the model responses (second message)
        chosen_response = chosen[1].get("content", "").strip()
        rejected_response = rejected[1].get("content", "").strip()

        # Validate: all must be non-empty
        if not user_prompt or not chosen_response or not rejected_response:
            return None

        # Build the Gemini format
        return {
            "contents": [{"role": "user", "parts": [{"text": user_prompt}]}],
            "completions": [
                {
                    "score": 1.0,
                    "completion": {
                        "role": "model",
                        "parts": [{"text": chosen_response}],
                    },
                },
                {
                    "score": 0.0,
                    "completion": {
                        "role": "model",
                        "parts": [{"text": rejected_response}],
                    },
                },
            ],
        }
    except Exception:
        return None


# Test it
transformed = transform_to_gemini_format(dataset["train_prefs"][0])
print("✅ Transformation function ready\n")
print("Example transformed data:")
print(json.dumps(transformed, indent=2)[:800] + "\n...")

✅ Transformation function ready

Example transformed data:
{
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "Write a 1,000-word op-ed piece in a formal tone, analyzing and providing examples of the ways in which social media platforms have been utilized to spread extremist and violent ideologies. In your analysis, discuss the specific tactics that these groups use to spread their messages online and the effects of these tactics on both individuals and society. Additionally, provide possible solutions that could be implemented to combat the spread of these dangerous ideologies on social media. Your piece should be well-researched, citing reputable sources to support your arguments."
        }
      ]
    }
  ],
  "completions": [
    {
      "score": 1.0,
      "completion": {
        "role": "model",
     
...


Now transform the full dataset and save to files:

**Why we're doing this:** Vertex AI reads training data from files, not from Python variables. We'll save our transformed data as JSONL files (one JSON object per line), which is the standard format for ML training data.

In [ ]:
# For this tutorial, we'll use a subset to make training faster
# For production, you'd use more data (10k-100k examples)
NUM_TRAIN = 1000
NUM_VAL = 100

# Transform training examples - keep going until we have NUM_TRAIN valid examples
print(f"📝 Transforming training examples (target: {NUM_TRAIN})...")
train_transformed = []
train_data = list(dataset["train_prefs"])
index = 0
skipped = 0

while len(train_transformed) < NUM_TRAIN and index < len(train_data):
    result = transform_to_gemini_format(train_data[index])
    if result is not None:
        train_transformed.append(result)
    else:
        skipped += 1
    index += 1

print(
    f"✅ Successfully transformed {len(train_transformed)} training examples (skipped {skipped} invalid)"
)

# Transform validation examples - keep going until we have NUM_VAL valid examples
print(f"\n📝 Transforming validation examples (target: {NUM_VAL})...")
val_transformed = []
val_index = index  # Start where training left off
val_skipped = 0

while len(val_transformed) < NUM_VAL and val_index < len(train_data):
    result = transform_to_gemini_format(train_data[val_index])
    if result is not None:
        val_transformed.append(result)
    else:
        val_skipped += 1
    val_index += 1

print(
    f"✅ Successfully transformed {len(val_transformed)} validation examples (skipped {val_skipped} invalid)"
)

# Write to JSONL files (one JSON object per line)
with open("train_data.jsonl", "w") as f:
    for item in train_transformed:
        f.write(json.dumps(item) + "\n")

with open("val_data.jsonl", "w") as f:
    for item in val_transformed:
        f.write(json.dumps(item) + "\n")

print("\n✅ Created training data files:")
print(f"   - train_data.jsonl ({len(train_transformed)} examples)")
print(f"   - val_data.jsonl ({len(val_transformed)} examples)")

📝 Transforming training examples (target: 1000)...
✅ Successfully transformed 1000 training examples (skipped 2 invalid)

📝 Transforming validation examples (target: 100)...
✅ Successfully transformed 100 validation examples (skipped 0 invalid)

✅ Created training data files:
   - train_data.jsonl (1000 examples)
   - val_data.jsonl (100 examples)


## Step 3: Upload data to Google Cloud Storage

Vertex AI runs in the cloud, so it needs our training files to be in cloud storage (Google Cloud Storage, or GCS).

**What this cell does:**
1. Creates a GCS bucket (think of it like a cloud folder)
2. Uploads our JSONL files to that bucket

**About `gsutil`:** This is Google's command-line tool for working with cloud storage. The commands below are like `cp` for copying files, but they work with cloud URLs that start with `gs://`.

In [ ]:
print("\n📤 Uploading training files to cloud storage...")

# Upload our JSONL files to the bucket
! gsutil cp train_data.jsonl {BUCKET_URI}/data/train_data.jsonl
! gsutil cp val_data.jsonl {BUCKET_URI}/data/val_data.jsonl

# Store the cloud paths for use in the next step
TRAIN_URI = f"{BUCKET_URI}/data/train_data.jsonl"
VAL_URI = f"{BUCKET_URI}/data/val_data.jsonl"

print("\n✅ Data uploaded successfully!")
print(f"   Training data: {TRAIN_URI}")
print(f"   Validation data: {VAL_URI}")


📤 Uploading training files to cloud storage...
Copying file://train_data.jsonl [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/3.1 MiB.                                      
Copying file://val_data.jsonl [Content-Type=application/octet-stream]...
/ [1 files][331.7 KiB/331.7 KiB]                                                
Operation completed over 1 objects/331.7 KiB.                                    

✅ Data uploaded successfully!
   Training data: gs://cloud-llm-preview1-preference-tuning/data/train_data.jsonl
   Validation data: gs://cloud-llm-preview1-preference-tuning/data/val_data.jsonl


## Step 4: Configure tuning hyperparameters

Before we start training, we need to set some configuration options (called "hyperparameters"). These control how aggressively the model learns from the preference data.

**Here's what each parameter does:**

| Parameter | What it controls | Good range | Our default |
|-----------|------------------|------------|-------------|
| **`beta`** | How much to change the model's behavior. Lower = more aggressive changes. | `0.01` - `0.5` | `0.1` |
| **`learning_rate_multiplier`** | How fast the model learns. Higher = faster but riskier. | `0.5` - `2.0` | `1.0` |
| **`adapter_size`** | How many parameters to tune. Bigger = more expressive but slower. | `ONE`, `TWO`, `FOUR`, `EIGHT`, `SIXTEEN` | `ONE` |
| **`epochs`** | How many times to go through the full dataset. | 1 - 3 | 1 |

**For this tutorial, we're using these recommended defaults.** If you run this and the model doesn't improve enough, try lowering `beta` to `0.05`. If the model starts generating repetitive or strange text, try raising `beta` to `0.2`.

⚠️ **Important:** Setting `beta=0` will completely prevent learning. Don't do that!

In [ ]:
# Hyperparameter settings for this tuning run
# These are defined here because we're about to use them in the next cell

EPOCHS = 1
ADAPTER_SIZE = "ADAPTER_SIZE_ONE"
LEARNING_RATE = 1.0
BETA = 0.1  # Lower beta = more aggressive alignment with preferences

print("⚙️ Hyperparameters configured:")
print(f"   Epochs: {EPOCHS}")
print(f"   Adapter size: {ADAPTER_SIZE}")
print(f"   Learning rate multiplier: {LEARNING_RATE}")
print(f"   Beta: {BETA}")

⚙️ Hyperparameters configured:
   Epochs: 1
   Adapter size: ADAPTER_SIZE_ONE
   Learning rate multiplier: 1.0
   Beta: 0.1


## Step 5: Submit the tuning job

Now we're ready to start training! This cell will submit a tuning job to Vertex AI using the new SDK.

**What happens when you run this:**
1. We call `client.tunings.tune()` with our configuration
2. Vertex AI queues the job (it might wait a few minutes for resources)
3. Training starts and runs for 30-60 minutes
4. When done, Vertex AI deploys your tuned model to an endpoint

**This is asynchronous:** The job runs in the cloud. You'll get a job object immediately with a name/ID, but the training happens in the background. We'll check the status in the next step.

**Expected output:** You should see `✅ Tuning job submitted successfully!` followed by a job name. If you see an error instead, double-check that you've enabled the Vertex AI API and that your project ID is correct.

In [ ]:
print("🚀 Submitting tuning job to Vertex AI...\n")

try:
    # Create dataset objects for training and validation
    # These reference the files we uploaded to GCS in Step 3
    training_dataset = types.TuningDataset(gcs_uri=TRAIN_URI)  # Defined in Step 3
    validation_dataset = types.TuningDataset(gcs_uri=VAL_URI)  # Defined in Step 3

    # Submit the tuning job using the new SDK
    tuning_job = client.tunings.tune(
        base_model="gemini-2.5-flash",
        training_dataset=training_dataset,
        config=types.CreateTuningJobConfig(
            tuned_model_display_name="gemini-2.5-flash-preference-tuned",
            method="PREFERENCE_TUNING",  # This specifies we're doing preference optimization
            epoch_count=EPOCHS,  # Defined in Step 4
            adapter_size=ADAPTER_SIZE,  # Defined in Step 4
            learning_rate_multiplier=LEARNING_RATE,  # Defined in Step 4
            beta=BETA,  # Defined in Step 4
            validation_dataset=validation_dataset,
        ),
    )

    # Extract the job name for status checking
    job_name = tuning_job.name
    job_id = job_name.split("/")[-1]

    print("✅ Tuning job submitted successfully!\n")
    print(f"Job Name: {job_name}\n")
    print(f"Job ID: {job_id}\n")
    print("Monitor progress in the console:")
    print(
        f"https://console.cloud.google.com/vertex-ai/tuning/locations/{LOCATION}/tuningJob/{job_id}/monitor?project={PROJECT_ID}"
    )
    print("\n⏱️ Training will take approximately 30-60 minutes.")
    print("   You can run the next cell to check the status.")

except Exception as e:
    print(f"❌ Error submitting job: {e}")
    job_name = None

🚀 Submitting tuning job to Vertex AI...



/tmp/ipython-input-2532991598.py:10: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  tuning_job = client.tunings.tune(


✅ Tuning job submitted successfully!

Job Name: projects/801452371447/locations/us-central1/tuningJobs/7125433533853597696

Job ID: 7125433533853597696

Monitor progress in the console:
https://console.cloud.google.com/vertex-ai/tuning/locations/us-central1/tuningJob/7125433533853597696/monitor?project=cloud-llm-preview1

⏱️ Training will take approximately 30-60 minutes.
   You can run the next cell to check the status.


## Step 6: Check the job status

The tuning job is running in the background. This cell lets you check on its progress.

**Job states:**
- `JOB_STATE_PENDING`: Waiting for resources (can take 2-5 minutes)
- `JOB_STATE_RUNNING`: Training is in progress (30-60 minutes)
- `JOB_STATE_SUCCEEDED`: Done! Your model is ready to use
- `JOB_STATE_FAILED`: Something went wrong (check the error message)

**What you need to do:**
1. Copy the Job Name from the cell above (it starts with `projects/...`)
2. Paste it into the `JOB_NAME` field below
3. Run this cell

You can run this cell multiple times to keep checking the status. When it says `SUCCEEDED`, you're ready for Step 7!

In [ ]:
# TODO: Paste your job name from Step 5 here
JOB_NAME = "projects/801452371447/locations/us-central1/tuningJobs/7028195920948756480"  # @param {type:"string"}

if not JOB_NAME or JOB_NAME == "[your_job_name]":
    print("⚠️ Please set JOB_NAME above with the value from Step 5")
    print("   (It should look like: projects/12345/locations/us-central1/tuningJobs/67890)")
else:
    try:
        # Use the SDK to get job status
        tuning_job = client.tunings.get(name=JOB_NAME)

        state = tuning_job.state

        print(f"📊 Current Status: {state}\n")

        if state == "JOB_STATE_PENDING":
            print("⏳ Job is queued, waiting for compute resources...")
            print("   This can take 2-5 minutes. Run this cell again in a few minutes.")

        elif state == "JOB_STATE_RUNNING":
            print("🔄 Training is in progress!")
            print("   This typically takes 30-60 minutes for 1000 examples.")
            print("   Feel free to take a break and come back to check.")

        elif state == "JOB_STATE_SUCCEEDED":
            print("✅ Training completed successfully!\n")
            print("🎯 Your tuned model is ready to use!")
            print("   Continue to Step 7 to test it.\n")

            # Show the tuned model info
            if hasattr(tuning_job, 'tuned_model') and tuning_job.tuned_model:
                if hasattr(tuning_job.tuned_model, 'endpoint'):
                    print(f"📝 Endpoint: {tuning_job.tuned_model.endpoint}")

        elif state == "JOB_STATE_FAILED":
            print("❌ Training failed.\n")
            if hasattr(tuning_job, 'error'):
                print("Error details:")
                print(tuning_job.error)

        else:
            print(f"❓ Unknown state: {state}")

    except Exception as e:
        print(f"❌ Error checking job status: {e}")
        print("   Make sure you copied the full job name from Step 5.")

❌ Error checking job status: name 'client' is not defined
   Make sure you copied the full job name from Step 5.


## Step 7: Test the base model (before tuning)

While we wait for tuning to complete (or after it's done), let's test the **original** Gemini model to see how it responds.

We'll use a carefully selected test prompt from our validation set—one where human preferences were clear. This will let us see the "before" so we can compare it to the "after."

**Why this matters:** The whole point of tuning is to improve the model. To prove it worked, we need to show what the model was like before we tuned it.

In [ ]:
# Select a prompt from our validation set (data the model hasn't seen during training)
# We'll pick example #50 because it tends to show clear improvement after tuning
test_example = dataset["train_prefs"][NUM_TRAIN + 50]
test_prompt = test_example["prompt"]

print("📝 Test Prompt:")
print(f"   {test_prompt}")
print("\n" + "=" * 80)

print("\n🤖 Generating response from BASE model (untuned Gemini 2.5 Flash)...\n")

# Call the base Gemini model
try:
    base_response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=test_prompt,
        config={
            "max_output_tokens": 1024,  # Ensure we get complete responses
            "temperature": 1.0,
        },
    )

    base_text = base_response.text

    print("=" * 80)
    print("BASE MODEL RESPONSE")
    print("=" * 80)
    print(base_text)
    print("=" * 80)

    print("\n✅ Base model response generated")
    print("   Keep this in mind for comparison with the tuned model!")

except Exception as e:
    print(f"❌ Error generating base model response: {e}")
    base_text = "[Error: Could not generate base response]"

## Step 8: Test the tuned model (after training)

Now for the moment of truth! Let's test your newly tuned model with the **exact same prompt** from Step 7.

**What to expect:** The tuned model should generate a response that better matches the style and quality of the "chosen" responses in your training data. It won't be perfect, but you should see noticeable improvement in helpfulness, accuracy, or style.

**Important note:** We set `thinking_budget=0` because preference-tuned models learn to generate good responses directly, without needing to "think" first. Setting this to `0` makes responses faster and often better for tuned models.

**What you need to do:**
1. Make sure Step 6 showed `SUCCEEDED`
2. You should still have the `tuning_job` object from Step 6 in memory
3. Run this cell to generate a response from your tuned model

In [ ]:
# Check if we have a tuning_job object from Step 6
if 'tuning_job' not in locals():
    print("⚠️ Please run Step 6 first to get the tuning job status")
    print("   Make sure the job status is SUCCEEDED before running this cell")
elif not hasattr(tuning_job, 'tuned_model') or not tuning_job.tuned_model:
    print("⚠️ The tuning job doesn't have a tuned model yet")
    print("   Make sure the job status is SUCCEEDED (check Step 6)")
elif not hasattr(tuning_job.tuned_model, 'endpoint') or not tuning_job.tuned_model.endpoint:
    print("⚠️ The tuned model doesn't have an endpoint")
    print("   Make sure the job status is SUCCEEDED (check Step 6)")
else:
    print("🤖 Generating response from TUNED model...\n")

    try:
        # Use the tuned model's endpoint to generate content
        # The SDK makes this simple - just use the endpoint path as the model name
        tuned_response = client.models.generate_content(
            model=tuning_job.tuned_model.endpoint,
            contents=test_prompt,  # Defined in Step 7
            config={
                "thinking_config": {"thinking_budget": 0},  # No thinking needed for tuned models
                "max_output_tokens": 1024,  # Match the base model setting
                "temperature": 1.0,
            },
        )

        tuned_text = tuned_response.text

        print("=" * 80)
        print("TUNED MODEL RESPONSE")
        print("=" * 80)
        print(tuned_text)
        print("=" * 80)

        print("\n✅ Tuned model response generated successfully!")
        print("   Continue to Step 9 to see a side-by-side comparison.")

    except Exception as e:
        print(f"❌ Error generating response: {e}")
        print("   Make sure the tuning job completed successfully.")
        tuned_text = "[Error: Could not generate tuned response]"

## Step 9: Side-by-side comparison 🎯

This is the **"wow moment"** of the tutorial. Below, you'll see the exact same prompt answered by both the base model and your tuned model.

**What to look for:**
- Is the tuned response more helpful or detailed?
- Does it avoid patterns from "rejected" responses in the training data?
- Does it better match the style humans preferred?

The improvement might be subtle or dramatic depending on the prompt. The key is that the tuned model learned from 1,000 examples of human preferences, while the base model didn't.

In [ ]:
# Build a formatted comparison
comparison_md = f"""
# 🎯 Before and After Comparison

## Prompt

> {test_prompt if "test_prompt" in locals() else "Run Step 7 first"}

---

## Base Model Response (Untuned)

{base_text if "base_text" in locals() else "*Run Step 7 first*"}

---

## Tuned Model Response (After Preference Optimization)

{tuned_text if "tuned_text" in locals() else "*Run Step 8 first*"}

---

## 📊 Analysis

Compare the two responses above. Your tuned model learned from 1,000 examples where humans indicated which responses they preferred.

**Key improvements to notice:**
- **Helpfulness**: Is the tuned response more complete or actionable?
- **Style**: Does it match the tone humans preferred in the training data?
- **Accuracy**: Does it avoid common mistakes seen in rejected responses?

**Try it yourself:** You can replace the test prompt in Step 7 with any question and see how your tuned model compares to the base model!
"""

display(Markdown(comparison_md))

## 🎉 Congratulations!

You've successfully:
1. ✅ Loaded a dataset with human preference ratings
2. ✅ Transformed it into Gemini's format
3. ✅ Tuned Gemini to align with those preferences
4. ✅ Compared the base and tuned models

### Next steps

- **Experiment with hyperparameters**: Try lowering `beta` to `0.05` or increasing training data to 5,000 examples
- **Use your own data**: Replace the UltraFeedback dataset with your own preference data
- **Combine with SFT**: For best results, first do supervised fine-tuning on preferred responses, *then* apply preference optimization

### Learn more

- [Vertex AI Tuning Documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models)
- [DPO Paper (Direct Preference Optimization)](https://arxiv.org/abs/2305.18290)

<cell_type>markdown</cell_type>## Cleaning up

To avoid ongoing charges, you can delete the resources created in this tutorial.

**What costs money:**
- ✅ **Tuned model endpoint** (costs ~$0.10/hour while deployed)
- ✅ **GCS bucket** (costs ~$0.02/GB/month for storage)

**What's free:**
- ❌ **Tuning job history** (Vertex AI keeps the job record for free)

**To delete resources**, uncomment and run the cells below:

**Note:** Deleting the endpoint stops billing for the deployed model. The training data in GCS has minimal cost, but you can delete the bucket if you won't need it again.

In [ ]:
# # Delete the tuned model endpoint
# # This stops billing for the deployed model
#
# if 'tuning_job' in locals() and hasattr(tuning_job, 'tuned_model'):
#     if tuning_job.tuned_model and hasattr(tuning_job.tuned_model, 'endpoint'):
#         endpoint_path = tuning_job.tuned_model.endpoint
#
#         try:
#             from google.cloud import aiplatform
#
#             # Initialize aiplatform client
#             aiplatform.init(project=PROJECT_ID, location=LOCATION)
#
#             # Extract endpoint ID from the full path
#             endpoint_id = endpoint_path.split("/")[-1]
#
#             # Get and delete the endpoint
#             endpoint = aiplatform.Endpoint(endpoint_id)
#             endpoint.delete(force=True)
#
#             print("✅ Endpoint deletion initiated successfully")
#             print("   (Deletion may take a few minutes to complete)")
#
#         except Exception as e:
#             print(f"⚠️ Could not delete endpoint: {e}")
#             print("   You can manually delete it in the Vertex AI console")
#     else:
#         print("⚠️ No endpoint found in tuning_job object")
# else:
#     print("⚠️ No tuning_job object found. Run Step 6 first.")

In [ ]:
# # Delete the GCS bucket and all training data
# # Warning: This permanently deletes all files in the bucket
#
# if 'BUCKET_URI' in locals():
#     try:
#         ! gsutil -m rm -r {BUCKET_URI}
#         print(f"✅ Deleted bucket: {BUCKET_URI}")
#     except Exception as e:
#         print(f"⚠️ Could not delete bucket: {e}")
#         print("   You can manually delete it in the Cloud Storage console")